In [ ]:
import pandas as pd

In [ ]:
dataset_hardware = pd.read_csv('/content/Hardware (1).csv')
dataset_software = pd.read_csv('/content/Copy_of_Software.csv')
dataset_os = pd.read_csv('/content/Operating_Systems.csv')

In [ ]:
dataset_os.head()

,Operating System,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Category
0,Windows 1.0 (1985),NaN,NaN,NaN,NaN,Windows Operating system
1,Windows 2.0 (1987),NaN,NaN,NaN,NaN,Windows Operating system
2,Windows 3.0 (1990),NaN,NaN,NaN,NaN,Windows Operating system
3,Windows 3.1 (1992),NaN,NaN,NaN,NaN,Windows Operating system
4,Windows for Workgroups 3.11 (1993),NaN,NaN,NaN,NaN,Windows Operating system


In [ ]:
dataset_os.columns

Index(['Operating System', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3',
       'Unnamed: 4', 'Category'],
      dtype='object')

In [ ]:
dataset_hardware = dataset_hardware['Name']

In [ ]:
dataset_software = dataset_software['Name']

In [ ]:
dataset_os = dataset_os['Name']

In [ ]:
dataset_os.rename(columns={'Operating System': 'Name'}, inplace=True)

In [ ]:
dataset_software['Category'] = 'Software'

In [ ]:
dataset_software.head()

0                                             uTorrent
1                                        Google Chrome
2             WhatsApp Messenger 64-bit for PC Windows
3                                        WinRAR 64-bit
4    2007 Microsoft Office Add-in: Microsoft Save a...
Name: Name, dtype: object

In [ ]:
dataset_os.head()

0                    Windows 1.0 (1985)
1                    Windows 2.0 (1987)
2                    Windows 3.0 (1990)
3                    Windows 3.1 (1992)
4    Windows for Workgroups 3.11 (1993)
Name: Name, dtype: object

In [ ]:
dataset_os.head()

0                    Windows 1.0 (1985)
1                    Windows 2.0 (1987)
2                    Windows 3.0 (1990)
3                    Windows 3.1 (1992)
4    Windows for Workgroups 3.11 (1993)
Name: Name, dtype: object

In [ ]:
data_final = pd.concat([dataset_hardware, dataset_os, dataset_software], ignore_index=True)

In [ ]:
data_final.to_csv('data_final.csv', index=False)

In [ ]:
dataset_os.to_csv('data_final_os.csv', index=False)

In [ ]:
dataset_software.head

<bound method NDFrame.head of 0                                                uTorrent
1                                           Google Chrome
2                WhatsApp Messenger 64-bit for PC Windows
3                                           WinRAR 64-bit
4       2007 Microsoft Office Add-in: Microsoft Save a...
                              ...                        
7191                                  Crowdstrike Falcone
7192                                              Radware
7193                                           Cloudflare
7194                                             Fortinet
7195                                           Zimbramail
Name: Name, Length: 7196, dtype: object>

In [ ]:
dataset_hardware = pd.read_csv('/content/data_final_hardware.csv')
dataset_software = pd.read_csv('/content/data_final_software.csv')
dataset_os = pd.read_csv('/content/data_final_os.csv')

In [ ]:
dataset_os.info

<bound method DataFrame.info of                                    Name Category
0                    Windows 1.0 (1985)       Os
1                    Windows 2.0 (1987)       Os
2                    Windows 3.0 (1990)       Os
3                    Windows 3.1 (1992)       Os
4    Windows for Workgroups 3.11 (1993)       Os
..                                  ...      ...
248                         IBM 4690 OS       Os
249                              ProDOS       Os
250                             PTS-DOS       Os
251                            TurboDOS       Os
252                             FreeDOS       Os

[253 rows x 2 columns]>

In [ ]:
data_final = pd.concat([dataset_hardware, dataset_os, dataset_software], ignore_index=True)

In [ ]:
data_final

<bound method DataFrame.info of                                                     Name  Category
0      Cisco 4000 Series Integrated Serv+A2:A41ices R...  Hardware
1            Cisco Catalyst IR1100 Rugged Series Routers  Hardware
2                                          Cisco Routers  Hardware
3      Cisco 1000 Series Aggregation Services Routers...  Hardware
4      Cisco 1000 Series Integrated Services Routers ...  Hardware
...                                                  ...       ...
12201                                Crowdstrike Falcone  Software
12202                                            Radware  Software
12203                                         Cloudflare  Software
12204                                           Fortinet  Software
12205                                         Zimbramail  Software

[12206 rows x 2 columns]>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
X = data_final['Name']  # Features (text)
y = data_final['Category']

In [ ]:
data_final['Name'].fillna('', inplace=True)
data_final['Category'].fillna('', inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)

In [ ]:
max_seq_length = 100
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_seq_length)
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_seq_length)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_seq_length))
model.add(LSTM(128))
model.add(Dense(3, activation='softmax'))  # 3 output classes (OS, Software, Hardware)

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [ ]:
# One-hot encode labels
from tensorflow.keras.utils import to_categorical

y_train_encoded = to_categorical(y_train_encoded, num_classes=3)
y_test_encoded = to_categorical(y_test_encoded, num_classes=3)


In [ ]:
model.fit(X_train_padded, y_train_encoded, epochs=10, batch_size=64, validation_data=(X_test_padded, y_test_encoded))

In [ ]:
y_pred = model.predict(X_test_padded)
y_pred_classes = np.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test_encoded, y_pred_classes)
print("Accuracy: ", accuracy)
print(classification_report(y_test_encoded, y_pred_classes))

In [ ]:
new_article = ""
new_article_sequence = tokenizer.texts_to_sequences([new_article])
new_article_padded = pad_sequences(new_article_sequence, maxlen=max_seq_length)
predicted_category = model.predict(new_article_padded)
predicted_category_label = label_encoder.inverse_transform([np.argmax(predicted_category)])
print("Predicted Category: ", predicted_category_label)

1/1 [==============================] - 0s 25ms/step
Predicted Category:  ['Software']
